# Cleaning and Wrangling: Seattle Open Street Map

In this project, we analyze Open Street Map for the City of Seattle.

## Explore a Subset of Data

Due to the size of the dataset, we need a way to systematically slice the original dataset for a workable sample to explore. To this end, I have used the following code to achieve this. The **k** value is changed from large to small so that my resulting 
*SAMPLE_FILE* ends up at different sizes. When starting out, try using a larger k, then move on to an intermediate k before processing your whole dataset.

In [1]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow
import xml.etree.cElementTree as ET
import pprint
import pickle
from collections import defaultdict
import re

In [2]:
OSM_FILE = "seattle_washington.osm"  # Replace this with your osm file
SAMPLE_FILE = "test.osm"

In [3]:
k = 5000 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

At the end of the above code, we end up with a file *test.osm* with which we can use to explore the dataset. 

## Develop a Dictionary for All Tags In the Original Dataset

Our goal here is to end up with a Python dictionary for the tags in the original dataset, so that we know what needs to be wrangled in the data. The following achives this.

In [4]:
def count_tags(filename):
    tags = {}
    
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    
    return tags

In [5]:
tags = count_tags('seattle_washington.osm')

In [6]:
with open('tags.pickle', 'wb') as tagsPickle:
    pickle.dump(tags, tagsPickle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
with open('tags.pickle', 'rb') as tagsPickle:
    unserialized_tags = pickle.load(tagsPickle)

In [7]:
unserialized_tags

{'bounds': 1,
 'member': 88068,
 'nd': 8453162,
 'node': 7580046,
 'osm': 1,
 'relation': 9411,
 'tag': 4708553,
 'way': 750242}

## Exploring What Is Contained Within Each Tag Type

To get a better sense of what sort of attributes is contained inside each type of tag, we use the following code to return this information to us.

In [8]:
bounds_subtags = []
member_subtags = []
nd_subtags = []
node_subtags = []
osm_subtags = []
relation_subtags = []
tag_subtags = []
way_subtags = []

for _, element in ET.iterparse('seattle_washington.osm'):
    if element.tag == 'bounds' and element.attrib.keys() not in bounds_subtags:
        bounds_subtags.append(element.attrib.keys())
    elif element.tag == 'member' and element.attrib.keys() not in member_subtags:
        member_subtags.append(element.attrib.keys())
    elif element.tag == 'nd' and element.attrib.keys() not in nd_subtags:
        nd_subtags.append(element.attrib.keys())
    elif element.tag == 'node' and element.attrib.keys() not in node_subtags:
        node_subtags.append(element.attrib.keys())
    elif element.tag == 'osm' and element.attrib.keys() not in osm_subtags:
        osm_subtags.append(element.attrib.keys())
    elif element.tag == 'relation' and element.attrib.keys() not in relation_subtags:
        relation_subtags.append(element.attrib.keys())
    elif element.tag == 'tag' and element.attrib.keys() not in tag_subtags:
        tag_subtags.append(element.attrib.keys())
    elif element.tag == 'way' and element.attrib.keys() not in way_subtags:
        way_subtags.append(element.attrib.keys())
    else:
        pass

In [9]:
bounds_subtags

[dict_keys(['maxlon', 'minlat', 'maxlat', 'minlon'])]

In [10]:
member_subtags

[dict_keys(['type', 'role', 'ref'])]

In [11]:
nd_subtags

[dict_keys(['ref'])]

In [12]:
node_subtags

[dict_keys(['lat', 'user', 'uid', 'id', 'timestamp', 'lon', 'changeset', 'version']),
 dict_keys(['lat', 'id', 'timestamp', 'lon', 'changeset', 'version'])]

In [13]:
osm_subtags

[dict_keys(['timestamp', 'generator', 'version'])]

In [14]:
relation_subtags

[dict_keys(['id', 'user', 'changeset', 'timestamp', 'uid', 'version'])]

In [15]:
way_subtags

[dict_keys(['id', 'user', 'changeset', 'timestamp', 'uid', 'version'])]

In [16]:
tag_subtags

[dict_keys(['k', 'v'])]

Based on the above, we started to get a sense of that the *xml* document was indeed used as a data storage tool, in that each of the elements in the above lists represents an **attribute** of an tag. Our data wrangling goal then, is to transform the information embedded in the OSM xml document into a JSON document with a flexible schema. In general, this schema will reflect the Python data structure of a dictionary, where an **attribute** is used as a **key**, and the **value of the attribute** is th corresponding **value** of the dictionary. 

Since a JSON document can be arbitrarily complex, it is an excellent fit for MongoDB, whose greatest feature is its flexible schema. 

## Problems Encountered in Seattle OSM

After generating several smallish subsets of Metro Seattle open street map xml files, I noticed the following problems with the data:

* Inconsistent representation of street name (original name and abbreviated names):
  Examples of this problem are King George Boulevard and also be written as King George Blvd, and Granville Street can also show up as Granville St or Granille St.

* Inconsistent representation of addresses under the **tag** tag:
  Address can either be represented by a many subtags, each with a key of **k='addr:city'**, **k='addr:street'**, **k='addr:postcode'**, **k='addr:housenumber'**. However, at times, an address can also be expressed as **k='addr:'**. 
  
We now proceed with the address cleaning.

## Audit Plan: Addresses

From a visual inspection of the subset of Seattle OSM, we understood that **tag** contains address information. In particular, tags with attribute of  **k** of **addr:street** contains street names that tend to be described inconsistently in the dataset. Therefore, our next goal is to develop a data audit plan that works specifically on tags with addresses.

The following chuncks of code achive this goal.

In [17]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [18]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [19]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [20]:
def audit(osmfile):
    osm_file = open('seattle_washington.osm', 'r', encoding='cp1252', errors='replace')
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [ ]:
st_types = audit(OSM_FILE)

The above Python dictionary shows the entire collection of street types after we have done our initial cleaning. Now we see that a vast majority of street types no longer bears problems. However, some of the street types are obviously wrong. Most notably, whenever **Suite number / apartment number** is present in the street name, the code has confused it with the name of the street. This needs our attention.

To address this problem, we should make sure that our subsequent code to clean and wrangle the OSM data will shape the raw data in such a way that will avoid confusing the suite number / apartment number with the street name. A convenient way to achieve this is to present an address in the JSON document (namely, the cleaned file) with schema such as this:

"address": {"street": "3401 Evanston Ave N, Suite A"}

In [1]:
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Ave.": "Avenue"
          }

In [15]:
def update_name(name, mapping):
    m = street_type_re.search(name)
    street_type = m.group()
    
    name = re.sub(street_type, mapping[street_type], name)
    return name

In [16]:
mapping

{'Ave': 'Avenue', 'Rd.': 'Road', 'St': 'Street', 'St.': 'Street'}

## Preparing the Data for Dababase Insertion

We can describe our audit plan so far as this:

* First we explored the variations and types of street name representation found in the Seattle OSM data. Then we normalized the abbreviated street type representations.

* Second transform the key-value pairs found in the **tag** tag as described earlier.

* Finally, we will normalize the overall address representation by removing the tags with attribute **addr:**, and make sure that our final **street** key will show the street name as well as the suite / apartment number.

After running the code below, then we are ready to insert the cleaned and wrangled data into the MongoDB database.

In [33]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import collections

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    position_attributes = ['lat', 'lon']
    created_attributes = CREATED

    if element.tag == "node" or element.tag == "way":
        # populate tag type
        node['type'] = element.tag

        # initialize address
        address = {}

        # parse through attributes
        for attribute in element.attrib:
            if attribute in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][attribute] = element.get(attribute)
            elif attribute in position_attributes:
                continue
            else:
                node[attribute] = element.get(attribute)

        # populate position
        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]

        # parse second-level tags for nodes
        for child in element:
            # parse second-level tags for ways and populate `node_refs`
            if child.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in child.attrib:
                    node['node_refs'].append(child.get('ref'))

            # throw out not-tag elements and elements without `k` or `v`
            if child.tag != 'tag'\
            or 'k' not in child.attrib\
            or 'v' not in child.attrib:
                continue
            key = child.get('k')
            val = child.get('v')

            # skip problematic characters
            if problemchars.search(key):
                continue

            # parse address k-v pairs
            elif address_regex.search(key):
                key = key.replace('addr:', '')
                address[key] = val


            # catch-all
            else:
                node[key] = val
        # compile address
        if len(address) > 0:
            node['address'] = {}
            street_full = None
            street_dict = {}
            street_format = ['prefix', 'name', 'type']
            # parse through address objects
            for key in address:
                val = address[key]
                if street_regex.search(key):
                    if key == 'street':
                        street_full = update_name(val, mapping) if val in mapping else val
                    elif 'street:' in key:
                        street_dict[key.replace('street:', '')] = update_name(val, mapping) if val in mapping else val
                else:
                    node['address'][key] = update_name(val, mapping) if val in mapping else val
            # assign street_full or fallback to compile street dict
            if street_full:
                node['address']['street'] = update_name(street_full, mapping) if street_full in mapping else street_full
            elif len(street_dict) > 0:
                node['address']['street'] = ' '.join([street_dict[key] for key in street_format])
        return node
    else:
        return None

In [34]:
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [35]:
data = process_map('seattle_washington.osm')

## Inserting Into MongoDB Database

After this step, we then insert into the MongoDB database using the following cmd command:

```
mongoimport -host 127.0.0.1:27017 --db osm --collection seattle_osm --drop --file C:\Users\Jenny\Documents\Mathfreak_Data\School\Data_Analysis_ND\Project3\seattle_washingotn.osm.json
```

## Exploring the Database in MongoDB

### File Size

```
seattle_washingotn.osm ......... 1,649 MB
seattle_washingotn.osm.json .... 1,870 MB
```

### Overview of Seattle Area Map

In [2]:
from pymongo import MongoClient

def get_db():
    client = MongoClient('localhost:27017')
    db = client.osm
    return db

In [3]:
db = get_db()
db.seattle_osm.find().count()      

8330288

We see that once cleaned and imported, the Seattle OSM collection has approximately 8.3 million data points.

In [4]:
db.seattle_osm.find({"type":"node"}).count()

7580018

We see that the Seattle OSM collection has roughly 7.5 million nodes.

In [5]:
db.seattle_osm.find({"type":"way"}).count()

750175

In [6]:
len(db.seattle_osm.distinct("created.user"))

3280

### Top 10 Amenities in Metro Seattle 

In [17]:
amenity_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity","count":{"$sum":1}}}, 
                          {"$sort":{"count":-1}}, {"$limit":10}])

In [18]:
for a in amenity_result:
    print(a)

{'count': 10753, '_id': 'parking'}
{'count': 3318, '_id': 'bicycle_parking'}
{'count': 3293, '_id': 'restaurant'}
{'count': 2848, '_id': 'bench'}
{'count': 2458, '_id': 'school'}
{'count': 1645, '_id': 'place_of_worship'}
{'count': 1578, '_id': 'fast_food'}
{'count': 1499, '_id': 'cafe'}
{'count': 1294, '_id': 'waste_basket'}
{'count': 1153, '_id': 'fuel'}


### Top 10 Place of Worship

In [19]:
religion_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
                                                {"$group":{"_id":"$religion", "count":{"$sum":1}}},
                                                {"$sort":{"count":-1}}, {"$limit":10}])

In [20]:
for r in religion_result:
    print(r)

{'count': 1500, '_id': 'christian'}
{'count': 80, '_id': None}
{'count': 20, '_id': 'jewish'}
{'count': 17, '_id': 'buddhist'}
{'count': 7, '_id': 'muslim'}
{'count': 6, '_id': 'unitarian_universalist'}
{'count': 3, '_id': 'sikh'}
{'count': 2, '_id': 'bahai'}
{'count': 2, '_id': 'eckankar'}
{'count': 2, '_id': 'spiritualist'}


### Top 10 Dining Choice

In [22]:
dinning_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}}, 
                          {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
                          {"$sort":{"count":-1}}, {"$limit":10}])

In [23]:
for d in dinning_result:
    print(d)

{'count': 817, '_id': None}
{'count': 265, '_id': 'mexican'}
{'count': 257, '_id': 'pizza'}
{'count': 249, '_id': 'american'}
{'count': 162, '_id': 'asian'}
{'count': 155, '_id': 'thai'}
{'count': 150, '_id': 'chinese'}
{'count': 123, '_id': 'japanese'}
{'count': 117, '_id': 'italian'}
{'count': 101, '_id': 'burger'}


### Top 10 Types of Coffee Shops

In [27]:
cafe_result = db.seattle_osm.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"cafe"}}, 
                          {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
                          {"$sort":{"count":-1}}, {"$limit":10}])

In [28]:
for c in cafe_result:
    print(c)

{'count': 663, '_id': 'coffee_shop'}
{'count': 567, '_id': None}
{'count': 48, '_id': 'ice_cream'}
{'count': 20, '_id': 'sandwich'}
{'count': 19, '_id': 'american'}
{'count': 15, '_id': 'tea'}
{'count': 14, '_id': 'donut;coffee_shop'}
{'count': 10, '_id': 'vietnamese'}
{'count': 10, '_id': 'donut'}
{'count': 9, '_id': 'frozen_yogurt'}


### Other Ideas

### Number of Distinct Points Contained in Map

Although the above data view showed that there are oughly 8.3 million data points, it does not directly address the question of how many distinct geographical points does Seattle OSM contain. To answer this question, we go about the route of usign the geo position of latitue and longitude, together with the code below.

In [ ]:
def make_pipeline():
    pipeline = [ ]
    group = {'$group':{'_id':'$pos', 'uniq_count': { '$sum': 1 }}}
    sort = {'$sort':{'count':-1}}
    group1 = {'$count':'uniq_count'}
    
    for e in [group, sort, group1]:
        pipeline.append(e)
    
    return pipeline

In [ ]:
def aggregate(db, pipeline):
    return [doc for doc in db.seattle_osm.aggregate(pipeline)]

Note that given the size of our returned query result, we have to write the results into a collection, and iterate through each element to get the detailed results contained withitn.

In [ ]:
pipeline = make_pipeline()
pos_result = db.seattle_osm.aggregate(pipeline, allowDiskUse=True)

In [ ]:
for p in pos_result:
    print(p)

We see that Seattle OSM contained roughly 7.5 million distinct geographical "points".

### Other Observations

I discovered that the Seatlte OSM data had inadvertently contained some points that actually does **not** belong to Seattle at all - in fact, they belong to a neighboring city in a neighboring country, Victoria, Canada. This was probably due to the works for GPS systems that categorized these points based on their vicinity to Seattle. Indeed, Victoria is closer to Seattle than to Vancouver, BC. 

This problem was discovered using this following exploration technique by looking at the name of the cities that were found in the dataset -- sorted in ascending order. While doing this exploration, all of the cities are indeed within Washington state, except for Victoria.

In [21]:
not_result = db.seattle_osm.aggregate([{"$match":{"address.city":{"$exists":1}}}, 
                   {"$group":{"_id":"$address.city", "count":{"$sum":1}}}, {"$sort":{"count":1}}])

In [22]:
for v in not_result:
    print(v)

{'count': 1, '_id': 'Rainier'}
{'count': 1, '_id': 'Port Angles'}
{'count': 1, '_id': 'Centalia'}
{'count': 1, '_id': 'everett'}
{'count': 1, '_id': 'Silvana'}
{'count': 1, '_id': 'Aberdeen, WA'}
{'count': 1, '_id': 'Roy'}
{'count': 1, '_id': 'Clyde Hill'}
{'count': 1, '_id': 'BOTHELL'}
{'count': 1, '_id': 'RENTON'}
{'count': 1, '_id': 'Picnic Point-North Lynnwood'}
{'count': 1, '_id': 'Greenbank (Whidbey Island)'}
{'count': 1, '_id': 'Tumwater, WA'}
{'count': 1, '_id': 'Cottage Lake'}
{'count': 1, '_id': 'McChord Field'}
{'count': 1, '_id': 'Graham'}
{'count': 1, '_id': 'Meridian'}
{'count': 1, '_id': 'Shelton, WA'}
{'count': 1, '_id': 'Olympia, WA '}
{'count': 1, '_id': 'redmond'}
{'count': 1, '_id': 'bainbridge Island'}
{'count': 1, '_id': 'Oting'}
{'count': 1, '_id': 'Newcastle, WA'}
{'count': 1, '_id': 'port Townsend'}
{'count': 1, '_id': 'Bothel'}
{'count': 1, '_id': 'Clearlake'}
{'count': 1, '_id': 'Yelm, WA'}
{'count': 1, '_id': 'Mt.Vernon;Mount Vernon'}
{'count': 1, '_id': 'go

A more direct way of finding out if this particular Canadian city was somehow included in the Seattle OSM data file, we use this folloing code block:

In [25]:
vic_result = db.seattle_osm.aggregate([{"$match":{"address.city":{"$eq":"Victoria"}}}, 
                   {"$group":{"_id":"$address.city", "count":{"$sum":1}}}, {"$sort":{"count":1}}])

In [26]:
for v in vic_result:
    print(v)

{'count': 729, '_id': 'Victoria'}


## Conclusion

After this review, it would appear that the Seattle OSM data set had leveraged some automatic geo-data collection techniques such as GPS devices while a vehicle is in motion. Although some of the datapoints clearly do not belong to Seattle metro area, I now feel that as a while, the Seatlte OSM has been cleaned sufficiently for the purposes of this exercise. 

As a further note of possible future research, I find the proper identification of whether a geo location shoudl belong in the Seattle area can be made better by taking advantage of some elementary classification techniques using machine learning. This might be a more applicable point as more and more geographical data are gradually being collected by automatic means in the future.